# DSC540 - Final Term Project Milestone 3
## Janine Par 

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

## Open the page and read using bs4
Working on two Wikipedia Websites

In [405]:
page1 = requests.get('https://en.wikipedia.org/wiki/List_of_restaurant_chains_in_the_United_States')
page2 = requests.get('https://en.wikipedia.org/wiki/List_of_Michelin_3-star_restaurants_in_the_United_States')


Create a BeautifulSoup object

In [406]:
soup_chain = BeautifulSoup(page1.text, 'html.parser')
soup_michelin = BeautifulSoup(page2.text, 'html.parser')

# (A) ***Work on Wikipedia Chain Restaurants***

In [407]:
print(type(soup_chain.contents))
print(soup_chain.contents)

<class 'list'>
['html', '\n', <html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of restaurant chains in the United States - Wikipedia</title>
<script>document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-fea

## 1. Find the section needed

In [408]:
# I have identified to use the wikitable table to get the right information for the GDP source
wiki_chain_dtls = soup_chain.find("div", {"class": 'mw-body-content mw-content-ltr'})
print(type(wiki_chain_dtls))
wiki_chain_dtls

<class 'bs4.element.Tag'>


<div class="mw-body-content mw-content-ltr" dir="ltr" id="mw-content-text" lang="en"><div class="mw-parser-output"><p>
The following is a <b>list of notable restaurant chains in the United States</b>.
</p>
<meta property="mw:PageProp/toc"/>
<h2><span id="Asian.2FPacific"></span><span class="mw-headline" id="Asian/Pacific">Asian/Pacific</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=List_of_restaurant_chains_in_the_United_States&amp;action=edit&amp;section=1" title="Edit section: Asian/Pacific">edit</a><span class="mw-editsection-bracket">]</span></span></h2>
<table class="wikitable sortable">
<tbody><tr>
<th>Name
</th>
<th>Original location
</th>
<th>Founded
</th>
<th>Headquarters
</th>
<th>Parent company
</th>
<th data-sort-type="number">Number of U.S. locations
</th>
<th>Areas served
</th>
<th>Notes
</th></tr>
<tr>
<td><a href="/wiki/Bonchon_Chicken" title="Bonchon Chicken">BonChon Chicken</a>
</td>
<td><a href="/wiki/Busan

## 2. Identify the Different types of tables which are defined in H3 and H2 tags

**2.1 Create a list of all H2**

In [409]:
heading_table = [h2 for h2 in wiki_chain_dtls.findAll('h2')]

In [410]:
#Create a list of each HR Rows inside the wikitable
h2_rows=[]
for h2 in wiki_chain_dtls.findAll('h2'):
    h2_rows.append(h2.text)
h2_rows

['Asian/Pacific[edit]',
 'Baked goods[edit]',
 'Beef[edit]',
 'Beverages[edit]',
 'Chicken[edit]',
 'Frozen desserts[edit]',
 'Hamburgers[edit]',
 'Hot dogs[edit]',
 'Mexican/Tex-Mex[edit]',
 'Pizza/Italian[edit]',
 'Salad/vegetarian/vegan[edit]',
 'Sandwiches[edit]',
 'Seafood[edit]',
 'Casual dining[edit]',
 'Arcade and game centers[edit]',
 'Defunct[edit]',
 'See also[edit]',
 'References[edit]']

**2.3 Create a list of all H3**

In [411]:
#Create a list of each HR Rows inside the wikitable
h3_rows=[]
for h3 in wiki_chain_dtls.findAll('h3'):
    h3_rows.append(h3.text)
h3_rows

['American[edit]',
 'Asian/Asian fusion[edit]',
 'Bar/brewpub[edit]',
 'Breakfast[edit]',
 'Caribbean[edit]',
 'Chicken[edit]',
 'European[edit]',
 'Hamburgers[edit]',
 'Mexican/Tex-Mex[edit]',
 'Italian[edit]',
 'Pizza[edit]',
 'Salad/vegan[edit]',
 'Seafood[edit]',
 'Southern/barbecue[edit]',
 'Steakhouse[edit]']

**2.4  Merge Two list**

In [412]:
heading_lists = h2_rows + h3_rows

**2.5 Clean and Remove Edit details**

In [413]:
for (i, item) in enumerate(heading_lists):
    heading_lists[i]=item.replace('[edit]', '') 
    print (i, heading_lists[i])
    
# these are the table lists

0 Asian/Pacific
1 Baked goods
2 Beef
3 Beverages
4 Chicken
5 Frozen desserts
6 Hamburgers
7 Hot dogs
8 Mexican/Tex-Mex
9 Pizza/Italian
10 Salad/vegetarian/vegan
11 Sandwiches
12 Seafood
13 Casual dining
14 Arcade and game centers
15 Defunct
16 See also
17 References
18 American
19 Asian/Asian fusion
20 Bar/brewpub
21 Breakfast
22 Caribbean
23 Chicken
24 European
25 Hamburgers
26 Mexican/Tex-Mex
27 Italian
28 Pizza
29 Salad/vegan
30 Seafood
31 Southern/barbecue
32 Steakhouse


**2.6 Re-order of List** 

We need to transform these lists values

13 Casual dining - Need to be remvoved as there are Sub-tables under Casual dining which are index 18-32

14 Arcade and game centers - Need to be pushed down

15 Defunct - Need to be removed, does not contain restaurant information

16 See also  - Need to be removed, does not contain restaurant information



In [414]:
#This re-order lists 

myorder = [0, 1, 2, 3, 4, 5,6,7,8,9,10,11,12,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,14,13,15,16]
heading_lists_new = [heading_lists[i] for i in myorder]

#display
for (i, item) in enumerate(heading_lists_new):
    print (i, heading_lists_new[i])

0 Asian/Pacific
1 Baked goods
2 Beef
3 Beverages
4 Chicken
5 Frozen desserts
6 Hamburgers
7 Hot dogs
8 Mexican/Tex-Mex
9 Pizza/Italian
10 Salad/vegetarian/vegan
11 Sandwiches
12 Seafood
13 American
14 Asian/Asian fusion
15 Bar/brewpub
16 Breakfast
17 Caribbean
18 Chicken
19 European
20 Hamburgers
21 Mexican/Tex-Mex
22 Italian
23 Pizza
24 Salad/vegan
25 Seafood
26 Southern/barbecue
27 Steakhouse
28 Arcade and game centers
29 Casual dining
30 Defunct
31 See also


In [415]:

'''Delese deletes these list values
29 Casual dining
30 Defunct
31 See also
''' 

del heading_lists_new[29:31]

# Display 
for (i, item) in enumerate(heading_lists_new):
    print (i, heading_lists_new[i])

0 Asian/Pacific
1 Baked goods
2 Beef
3 Beverages
4 Chicken
5 Frozen desserts
6 Hamburgers
7 Hot dogs
8 Mexican/Tex-Mex
9 Pizza/Italian
10 Salad/vegetarian/vegan
11 Sandwiches
12 Seafood
13 American
14 Asian/Asian fusion
15 Bar/brewpub
16 Breakfast
17 Caribbean
18 Chicken
19 European
20 Hamburgers
21 Mexican/Tex-Mex
22 Italian
23 Pizza
24 Salad/vegan
25 Seafood
26 Southern/barbecue
27 Steakhouse
28 Arcade and game centers
29 See also


# 3. Get Table for each Heading Type

**3.1 Create a table list**

In [416]:
table_dtl  = [table for table in wiki_chain_dtls.findAll('table')]
table_dtl = table_dtl[0:29] # Just get the first 28 tables
table_dtl

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Name
 </th>
 <th>Original location
 </th>
 <th>Founded
 </th>
 <th>Headquarters
 </th>
 <th>Parent company
 </th>
 <th data-sort-type="number">Number of U.S. locations
 </th>
 <th>Areas served
 </th>
 <th>Notes
 </th></tr>
 <tr>
 <td><a href="/wiki/Bonchon_Chicken" title="Bonchon Chicken">BonChon Chicken</a>
 </td>
 <td><a href="/wiki/Busan" title="Busan">Busan, South Korea</a>
 </td>
 <td>2002
 </td>
 <td><a class="mw-redirect" href="/wiki/Dallas,_Texas" title="Dallas, Texas">Dallas, Texas</a>
 </td>
 <td>
 </td>
 <td>115
 </td>
 <td>Nationwide
 </td>
 <td>
 </td></tr>
 <tr>
 <td><a href="/wiki/Cupbop" title="Cupbop">Cupbop</a>
 </td>
 <td><a href="/wiki/Salt_Lake_County,_Utah" title="Salt Lake County, Utah">Salt Lake County, Utah</a>
 </td>
 <td>2013
 </td>
 <td><a href="/wiki/Salt_Lake_City" title="Salt Lake City">Salt Lake City</a>, <a href="/wiki/Utah" title="Utah">Utah</a>
 </td>
 <td>
 </td>
 <td>38
 </td>
 <td>Mountain state

**3.2 Each sub tables have different columns and number of columns. This loop will**

a) for every table and in each tr(row), get the th and generate a list

b) Create a dictionary with Key as (table number) and Value as the list of TH




In [417]:
pattern = r'[\(\[].*?[\)\]]'
column_table=[] 

dict_col={}
for i, table_l in enumerate(table_dtl):
    test = [[re.sub(pattern, '', td.getText().replace('\n','')) for td in tr.findAll('th')] for tr  in table_dtl[i].findAll('tr')] 
    dict_col[i] = test
    
    

**c) See below that only the 1st list value [0] has th, because only the first row of each table indicate heading**

In [418]:
print(dict_col[0][0])
print(dict_col[1][0])
print(dict_col[2][0])
print(dict_col[3][0])

['Name', 'Original location', 'Founded', 'Headquarters', 'Parent company', 'Number of U.S. locations', 'Areas served', 'Notes']
['Name', 'Original location', 'Founded', 'Headquarters', 'Parent company', 'Number of U.S. locations', 'Areas served', 'Notes']
['Name', 'Original location', 'Founded', 'Headquarters', 'Number of U.S. locations', 'Areas served', 'Notes']
['Name', 'Original location', 'Founded', 'Headquarters', 'Number of U.S. locations', 'Areas served', 'Notes']


## 4. Generate dataframe for each tables in the website

***4.1 
Create an empty list body table
Create an dictionary dict_table - This will have the key (table heading from the list generated from step 2) and value of actual dataframe which the table data that matches the table heading)

4.1 For each table, get the TD in every Row TR and get text. 

4.2 Clean data as some has special character and new line

4.3 Exclude the first row in the list as this is the heading (body_table=body_table[2:]) 

4.3 Generate a dataframe using the body_table with column names that index number matches to the key value from column step 3.2c

4.4 Append the dataframe dict_table where key is the heading table type and value as the dataframe  


In [419]:
pattern = r'[\(\[].*?[\)\]]'
body_table=[] 
dict_table={}
for i, table_l in enumerate(table_dtl):
    body_table = [[re.sub(pattern, '', td.getText().replace('\n','')) for td in tr.findAll('td')] for tr  in table_dtl[i].findAll('tr')] 
    body_table=body_table[2:] 
    dict_table[heading_lists_new[i]] =pd.DataFrame(body_table, columns=dict_col[i][0])  #create dictionary of all the sub table dataframe
   

# 5. Final Wikipedia Chain Restaurants

In [420]:
for k, dataframe in dict_table.items():
    print('''----------Table: {}---------------------'''.format(k))
    print(dataframe.head(3))

----------Table: Asian/Pacific---------------------
                    Name              Original location Founded  \
0                 Cupbop         Salt Lake County, Utah    2013   
1  L&L Hawaiian Barbecue               Honolulu, Hawaii    1976   
2             Manchu Wok  Peterborough, Ontario, Canada    1980   

               Headquarters  Parent company Number of U.S. locations  \
0      Salt Lake City, Utah                                       38   
1          Honolulu, Hawaii                                      209   
2  Markham, Ontario, Canada  MTY Food Group                       14   

                                        Areas served  \
0                          Mountain states, Oklahoma   
1  Western states, Hawaii, Alaska, Texas, New Yor...   
2                                         Nationwide   

                              Notes  
0                                    
1  Known as L&L Drive-Inn in Hawaii  
2                                    
----------Tab

In [421]:
#See each dataframe in a dictionary
dict_table["Beef"]

,Name,Original location,Founded,Headquarters,Number of U.S. locations,Areas served,Notes
0,Shane's Rib Shack,"McDonough, Georgia",2002,"Atlanta, Georgia",74,Southeast,
1,Skyline Chili,"Cincinnati, Ohio",1949,"Fairfield, Ohio",160,"Great Lakes, Florida",


In [422]:
dict_table["Arcade and game centers"]

,Name,Original location,Founded,Headquarters,Number of U.S. locations,Areas served,Notes
0,Dave & Busters,"Dallas, Texas",1982,"Dallas, Texas",142,Nationwide,
1,Peter Piper Pizza,"Glendale, Arizona",1973,"Phoenix, Arizona",149,"West, Florida",
2,Topgolf,Watford,2000,"Dallas, Texas",70,"United States, United Kingdom, Australia, Cana...",


# (B) Work on Wikipedia Michelin

## 1. Find the section needed

In [423]:
soup_michelin

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of Michelin 3-star restaurants in the United States - Wikipedia</title>
<script>document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-featur

In [424]:
table_dtl  = [soup_michelin.find('table'), {"class": '"wikitable sortable plainrowheaders"'}]


## 2. Get the TD data, Clean the data and then Generate a list


In [425]:
body_table1 = [[re.sub(pattern, '', td.getText().replace('\n','')) for td in tr.findAll('td')] for tr  in table_dtl[0].findAll('tr')] 


## 3. Final Wikipedia Chain Restaurants

**3.1. Generate a column list
  3.2. Generate a dataframe
***


In [426]:
columns_names =['Restaurant', 'Chefs', 'Cuisine', 'Location','Year_Michelin_1Star','Year_Michelin_2Stars','Year_Michelin_3Star', 'Note','Ref']

df_michelin=pd.DataFrame(body_table1[2:],columns=columns_names )
df_michelin

,Restaurant,Chefs,Cuisine,Location,Year_Michelin_1Star,Year_Michelin_2Stars,Year_Michelin_3Star,Note,Ref
0,Alinea,Grant Achatz and Doug Alley,Molecular Gastronomy,Chicago,—,—,2011–present,,
1,SingleThread,Kyle Connaughton,Japanese and New American,"Healdsburg, California",—,2018,2019–present,,
2,Manresa,David Kinch and Nicholas Romero,California,"Los Gatos, California",—,2006–2015,2016–2022,Closed,
3,Le Bernardin,"Éric Ripert, Chris Muller, and Eric Gestel",French,New York City,—,—,2006–present,,
4,Per Se,Thomas Keller and Corey Chow,New American,New York City,—,—,2006–present,,
5,Masa,Masa Takayama,Japanese and sushi,New York City,—,2006–2008,2009–present,,
6,Eleven Madison Park,Daniel Humm and Brian Lockwood,Plant-based Contemporary American,New York City,2010–2011,—,2012–present,,
7,Chef's Table at Brooklyn Fare,César Ramírez,"Seafood, Omakase-style",New York City,—,2011,2012–present,,
8,Daniel,Daniel Boulud,French,New York City,—,2006–2010 2016–present,2011–2015,,
9,Jean-Georges,Mark LaPico,French,New York City,—,2018–present,2006–2017,,
